# BioExplorer - Neurons
![](../bioexplorer_neurons_banner.png)

### Connect to back-end

In [69]:
from bioexplorer import BioExplorer, Vector3

url = 'localhost:5000'
be = BioExplorer(url)

core = be.core_api()
status = be.reset_scene()

### Load neurons

In [70]:
assembly_name = 'Neurons'
be.remove_assembly(assembly_name)
neurons_assembly = be.add_assembly(assembly_name)

scale = Vector3(1, 1, 1)

neurons_model = be.add_neurons(
    assembly_name=assembly_name,
    population_name='o1',
    geometry_quality=be.GEOMETRY_QUALITY_MEDIUM,
    morphology_color_scheme=be.MORPHOLOGY_COLOR_SCHEME_SECTION,
    use_sdf=True, load_synapses=True, generate_buttons=True,
    load_somas=True, load_axon=True, show_membrane=True,
    load_basal_dendrites=True, load_apical_dendrites=True,
    generate_internals=True, generate_externals=False,
    sql_node_filter='guid=47211',
    scale=scale
)

In [71]:
be.reset_camera()
core.set_renderer()

True

In [72]:
import seaborn as sns
def set_morphology_materials(model_id, palette_name, palette_size):
    colors = list()
    opacities = list()
    refraction_indices = list()
    specular_exponents = list()
    shading_modes = list()
    user_params = list()
    glossinesses = list()
    emissions = list()
    
    material_ids = be.get_material_ids(model_id)['ids']
    palette = sns.color_palette(palette_name, palette_size)

    for material_id in material_ids:
        mid = material_id % palette_size
        color = palette[mid]
        if mid in [be.NEURON_MATERIAL_AFFERENT_SYNPASE, be.NEURON_MATERIAL_EFFERENT_SYNPASE]:
            # colors.append(color)
            colors.append(palette[0])
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(0.1)
            user_params.append(0.001 / scale.x)
            specular_exponents.append(5.0)
            emissions.append(0.0)
        elif mid == be.NEURON_MATERIAL_MITOCHONDRION:
            colors.append([1.0, 0.0, 1.0])
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(0.5)
            user_params.append(1.0 / scale.x)
            specular_exponents.append(6.0)
            emissions.append(0.25)
        elif mid == be.NEURON_MATERIAL_NUCLEUS:
            colors.append([1.0, 1.0, 1.0])
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(1.0)
            user_params.append(0.2 / scale.x)
            specular_exponents.append(30.0)
            emissions.append(0.0)
        elif mid == be.NEURON_MATERIAL_SOMA:
            # colors.append(color)
            colors.append(palette[0])
            opacities.append(0.75)
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(0.2)
            user_params.append(0.01 / scale.x)
            specular_exponents.append(5.0)
            emissions.append(0.0)
        elif mid == be.NEURON_MATERIAL_MYELIN_STEATH:
            colors.append([0.4, 0.3, 0.5])
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_DIFFUSE)
            glossinesses.append(1.0)
            user_params.append(1.0)
            specular_exponents.append(50.0)
            emissions.append(0.0)
        else:
            # Membrane
            # colors.append(color)
            colors.append(palette[0])
            opacities.append(1.0)
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(0.1)
            user_params.append(0.1 / scale.x)
            specular_exponents.append(5.0)
            emissions.append(0.0)
            
        refraction_indices.append(1.0)
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=colors, specular_colors=colors,
        opacities=opacities, refraction_indices=refraction_indices,
        shading_modes=shading_modes, specular_exponents=specular_exponents,
        user_parameters=user_params, glossinesses=glossinesses,
        emissions=emissions
    )

In [73]:
model_ids = be.get_model_ids()['ids']
offset = model_ids[0]
set_morphology_materials(offset, 'Set3', be.NB_MATERIALS_PER_MORPHOLOGY)
core.set_renderer()

True

In [74]:
status = core.set_renderer(
    background_color=[1,1,1],
    current='bio_explorer',subsampling=4, max_accum_frames=64)
params = core.BioExplorerRendererParams()
params.gi_distance = 10.0 * scale.x
params.shadows = 1.0
params.soft_shadows = 1.0
params.epsilon_factor = 40.0
params.max_bounces = 10
params.show_background = True
params.exposure = 1.1
status = core.set_renderer_params(params)
status = core.set_renderer()